In [282]:
import pandas as pd
import numpy as np
import itertools
import json
import copy

class LitBot:
    EPSILON = 0.01
    def __init__(self, team_id, player_id, player_count, game_state_array=None, fake_game_seed = None):

        self.team_id = team_id
        self.player_id = player_id
        self.player_count = player_count

        if fake_game_seed is not None:
            (
                self._fake_initial_game_state_array,
                self._fake_initial_card_location_array
            ) =  self.initialize_fake_game(fake_game_seed)
            self.initialize_matrix(self.team_id, self.player_id, self._fake_initial_game_state_array[0, 0])
        elif game_state_array is not None:
            self.initialize_matrix(self.team_id, self.player_id, game_state_array)
    
    def initialize_fake_game(self, fake_game_seed):
        
        np.random.seed(fake_game_seed)
        card_location_array = np.random.choice(range(self.player_count), (8,6))
        card_location_array_team = card_location_array%2
        card_location_array_player = card_location_array//2

        game_state_array = np.zeros((2, self.player_count//2, 8, 6))
        for set_id, card_id in itertools.product(range(8), range(6)):
            game_state_array[
                card_location_array_team[set_id, card_id],
                card_location_array_player[set_id, card_id],
                set_id,        
                card_id
            ] = 1
        
        return game_state_array, card_location_array
    
    def initialize_matrix(self, team_id, player_id, player_array):

        self.information_matrix = np.full((2, self.player_count//2, 8, 6), LitBot.EPSILON)
        for team_idx, player_idx in itertools.product(range(2), range(self.player_count//2)):
            self.information_matrix[team_idx, player_idx, :, :] = np.where(
                player_array == 1, 
                1, 
                0 if (team_idx == team_id) and (player_idx == player_id) else LitBot.EPSILON
            )

        self.player_set_card_count = np.full((2, self.player_count//2, 8), 0)
        self.player_set_card_count[team_id, player_id, :] =  player_array.sum(axis=1)
        
        self.player_card_count = np.full((2, self.player_count//2), 48//self.player_count)
                
        self.set_card_count = np.full((8), 6)

        self.active_sets = np.full((8), LitBot.EPSILON) # active 1 closed sets keep scores here
        self.active_cards = np.full((8,6), LitBot.EPSILON)
        self.recent_card_array = np.full((2,8), LitBot.EPSILON)
        
        self.inferences_list = {}  # inference will be stored as {active cards array copy : 1}

    def inv_team_id(self, team_id):
        return int(not bool(team_id))


    def update_information_matrix_hard(self, card_info, ask_player, ans_player, result):

        if result == 0:
            assert ans_player is not None
            print(f"updated information matrix for {card_info['set_id'], card_info['card_id']} for result 0 : hard")
            self.information_matrix[ask_player["team_id"], ask_player["player_id"], card_info["set_id"], card_info["card_id"]] = 0
            self.information_matrix[ans_player["team_id"], ans_player["player_id"], card_info["set_id"], card_info["card_id"]] = 0

        if result == 1:
            print(f"updated information matrix for {card_info['set_id'], card_info['card_id']} for result 1 : hard")
            self.information_matrix[:, :, card_info["set_id"], card_info["card_id"]] = 0
            self.information_matrix[ask_player["team_id"], ask_player["player_id"], card_info["set_id"], card_info["card_id"]] = 1
            if ans_player is not None:
                print(f"updated player card count matrix for {card_info['set_id'], card_info['card_id']} for result 1 : hard")
                self.player_card_count[ask_player["team_id"], ask_player["player_id"]] += 1
                self.player_card_count[ans_player["team_id"], ans_player["player_id"]] -= 1


    @property
    def scores(self):
        return np.array([3,2])

    def update_active_arrays(self, team_id, set_id, card_id, action, result):
        if action == "ask_card":
            assert card_id is not None
            if result == 0:
                self.active_cards[set_id, card_id] = 1
                self.recent_card_array[team_id, set_id] = card_id
            else:
                self.active_cards[set_id, card_id] = 0
            self.active_sets[set_id] = 1
        if action == "call_set":
            self.active_cards[set_id, :] = 0
            self.active_sets[set_id] = 0

    def update_player_card_count(self, card_info, ask_player, ans_player, result):
        pass        

    def update_inference_list(self, team_id, player_id, set_id, card_id):

        if (self.recent_card_array[team_id, set_id] != card_id) and (self.recent_card_array[team_id, set_id] != LitBot.EPSILON):
            print(f"new inference detected for {set_id, self.recent_card_array[team_id, set_id]} for player {team_id, player_id} : inference")
            new_inference = {
                "team_id" : team_id, 
                "player_id" : player_id, 
                "set_id" : set_id, 
                "card_id" : self.recent_card_array[team_id, set_id]
            }
        
            conflict_inference_ids = []
            for id, inference in self.inferences_list.items():
                if (inference["set_id"] != new_inference["set_id"]) or (inference["card_id"] != new_inference["card_id"]):
                    continue
                elif (inference["team_id"] == team_id) or (inference["player_id"] == player_id):
                    print(f"new inference detected is already captured : inference")
                    continue
                else:
                    print(f"conflict inference detected for {new_inference['set_id'], new_inference['card_id']} for player {team_id, player_id} : inference")
                    conflict_inference_ids.append(id)

            assert conflict_inference_ids.__len__()<=1
            if conflict_inference_ids.__len__() == 1:
                print(f"conflict inference removed for {new_inference['set_id'], new_inference['card_id']} for player {team_id, player_id} : inference")
                self.inferences_list.pop(conflict_inference_ids[0])
            
            if self.inferences_list.__len__() == 0:
                self.inferences_list[0] = new_inference
            else:
                self.inferences_list[np.array(list(self.inferences_list.keys())).max()+1] = new_inference

        # TODO
        # detect new inference conflict with info matrix
        # detect inference conflicts on information matrix
        # detect out of date inferences


    def update_game(self, game_action_dict, stop = True):

        action = game_action_dict["action"]
        team_id = game_action_dict["by_team"]
        player_id = game_action_dict["by"]
        set_id = game_action_dict["set_id"]
        result = game_action_dict["result"]
        card_id = None

        if action == "ask_card":
            ans_team_id = game_action_dict["to_team"]
            ans_player_id = game_action_dict["to"]
            card_id = game_action_dict["card_id"]

            self.update_information_matrix_hard(
                card_info = {"set_id" : set_id, "card_id" : card_id},
                ask_player = {"team_id" : team_id, "player_id" : player_id},
                ans_player = {"team_id" : ans_team_id, "player_id" : ans_player_id},
                result = result
            )
        elif action == "call_set":
            card_locations = game_action_dict["card_locations"]
            for card_id, player_id in card_locations.items():
                card_id = int(card_id)

                self.update_information_matrix_hard(
                    card_info = {"set_id" : set_id, "card_id" : card_id},
                    ask_player = {"team_id" : team_id, "player_id" : player_id},
                    ans_player = None,
                    result = 1
                )
        else:
            pass
        
        self.update_inference_list(team_id, player_id, set_id, card_id)
        self.update_active_arrays(team_id, set_id, card_id, action, result)
        # self.update_player_card_count(
        #     card_info = {"set_id" : set_id, "card_id" : card_id},
        #     ask_player = {"team_id" : team_id, "player_id" : player_id},
        #     ans_player = {"team_id" : ans_team_id, "player_id" : ans_player_id},
        #     result = result
        # )

    @property
    def inference_multiplier_array(self):

        inference_array = np.full((2, self.player_count//2, 8,6), 1)
        
        for id, inference in self.inferences_list.items():
            team_id = inference["team_id"]
            player_id = inference["player_id"]
            set_id = int(inference["set_id"])
            card_id = int(inference["card_id"])

            inference_array[:, :, set_id, card_id] = 0
            inference_array[team_id, player_id, set_id, card_id] = 1

        return inference_array

    @property
    def prob_matrix(self):

        info_matrix = np.where(self.information_matrix == LitBot.EPSILON, 1, self.information_matrix)
        integrated_info_matrix = info_matrix * self.inference_multiplier_array

        prob_matrix = integrated_info_matrix/integrated_info_matrix.sum(axis=(0,1))

        return prob_matrix
    
    @property
    def shannon_info_matrix(self):

        uncertainity_matrix = self.prob_matrix.max(axis=(0, 1))
        shannon_info_matrix = self.total_shannon_info + np.log2(uncertainity_matrix)

        return shannon_info_matrix
    
    @property
    def total_shannon_info(self):
        return -np.log2(np.full((8,6), 1/self.player_count))


In [283]:
with open("gameActionData.json") as f:
    action_data = json.loads(f.read())

In [295]:
bot1 = LitBot(
    1,
    1,
    action_data["player_count"],
    game_state_array=np.where(np.array(action_data["card_location_array"])==4, 1, 0)
)

In [296]:
import time

In [297]:
i = 0
while i<10:
    # time.sleep(1)
    print(action_data["actions"][str(i)])
    bot1.update_game(action_data["actions"][str(i)])
    i += 1   

{'action': 'ask_card', 'by_team': 1, 'by': 1, 'to_team': 0, 'to': 1, 'set_id': 3, 'card_id': 5, 'result': 0}
updated information matrix for (3, 5) for result 0 : hard
{'action': 'ask_card', 'by_team': 0, 'by': 1, 'to_team': 1, 'to': 2, 'set_id': 6, 'card_id': 1, 'result': 1}
updated information matrix for (6, 1) for result 1 : hard
updated player card count matrix for (6, 1) for result 1 : hard
{'action': 'ask_card', 'by_team': 0, 'by': 1, 'to_team': 1, 'to': 0, 'set_id': 6, 'card_id': 4, 'result': 0}
updated information matrix for (6, 4) for result 0 : hard
{'action': 'ask_card', 'by_team': 1, 'by': 0, 'to_team': 0, 'to': 0, 'set_id': 3, 'card_id': 5, 'result': 0}
updated information matrix for (3, 5) for result 0 : hard
{'action': 'ask_card', 'by_team': 0, 'by': 0, 'to_team': 1, 'to': 1, 'set_id': 6, 'card_id': 4, 'result': 0}
updated information matrix for (6, 4) for result 0 : hard
{'action': 'ask_card', 'by_team': 1, 'by': 1, 'to_team': 0, 'to': 2, 'set_id': 3, 'card_id': 5, 'resu